# Unsupervised Domain Adaptation Project


## Part-1: Data download
Load data to project from Google Drive. Copy a subset of classes of images to the path:
- `adaptiope_small/product_images`
- `adaptiope_small/real_life` 

two directories. They represent images from two different domain **product** and **real_life**

In [ ]:
from os import makedirs, listdir
from tqdm import tqdm
from google.colab import drive
from os.path import join
from shutil import copytree

drive.mount('/content/gdrive')

!mkdir dataset
!cp "gdrive/My Drive/Colab Notebooks/data/Adaptiope.zip" dataset/
# !ls dataset

!unzip -qq dataset/Adaptiope.zip   # unzip file

!rm -rf dataset/Adaptiope.zip 
!rm -rf adaptiope_small

Mounted at /content/gdrive


In [ ]:
!mkdir adaptiope_small
classes = listdir("Adaptiope/product_images")
print(classes)
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]
domain_classes = ["product_images", "real_life"]
for d, td in zip(["Adaptiope/product_images", "Adaptiope/real_life"], ["adaptiope_small/product_images", "adaptiope_small/real_life"]):
  makedirs(td)
  for c in tqdm(classes):
    c_path = join(d, c)
    c_target = join(td, c)
    copytree(c_path, c_target)

['bottle', 'file cabinet', 'brachiosaurus', 'binoculars', 'stethoscope', 'smartphone', 'mixing console', 'axe', 'purse', 'hard-wired fixed phone', 'golf club', 'pipe wrench', 'puncher', 'computer mouse', 'speakers', 'fighter jet', 'diving fins', 'knife', 'ladder', 'projector', 'glasses', 'laptop', 'tape dispenser', 'spatula', 'game controller', 'electric shaver', 'tank', 'ring binder', 'trash can', 'crown', 'magic lamp', 'printer', 'shower head', 'toothbrush', 'hair dryer', 'stapler', 'chainsaw', 'sleeping bag', 'motorbike helmet', 'wristwatch', 'vr goggles', 'phonograph', 'drum set', 'wheelchair', 'sword', 'bicycle', 'letter tray', 'network switch', 'ice cube tray', 'nail clipper', 'handcuffs', 'umbrella', 'coat hanger', 'hoverboard', 'tent', 'computer', 'flat iron', 'rubber boat', 'snow shovel', 'rifle', 'telescope', 'dart', 'acoustic guitar', 'corkscrew', 'ruler', 'scissors', 'watering can', 'scooter', 'fan', 'hot glue gun', 'hourglass', 'razor', 'calculator', 'wallet', 'cordless fi

100%|██████████| 20/20 [00:04<00:00,  4.57it/s]


## Part-2: Image Classification Neural Network

 

### Part-2.0: Data Loading

First we load the data and preprocessing them

In [ ]:
product_path = 'adaptiope_small/product_images'
real_life_path = 'adaptiope_small/real_life'

In [ ]:
!pwd
!ls

/content
Adaptiope  adaptiope_small  dataset  gdrive  sample_data


In [ ]:
from PIL import Image
from os.path import join

img = Image.open(join(product_path, 'backpack', 'backpack_003.jpg'))
print('Image size: ', img.size)
#img

Image size:  (679, 679)


import libraries

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import vgg11, alexnet 
from torch.utils.data import DataLoader, random_split

configuration constants

In [ ]:
img_size = 256
# mean, std used by pre-trained models from PyTorch
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
config = dict(epochs=5, batch_size=64,lr=0.001, wd=0.001, momentum=0.9, domain_regression_weight=0.3)

Configue GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
from torchvision.transforms.transforms import ToTensor

def get_dataset(root_path):
  '''
    Get dataset from specific data path

    # parameters:
        root_path: path to image folder

    # return: train_loader, test_loader
  '''
  # Construct image transform
  image_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
  ])

  # Load data from filesystem
  image_dataset = ImageFolder(root_path, transform=image_transform)

  return image_dataset

def get_dataloader(dataset, batch_size, shuffle_train=True, shuffle_test=False):
  '''
    Get DataLoader from specific data path

    # parameters:
        dataset: ImageFolder instance
        batch_size: batch_size for DataLoader
        shuffle_train: whether to shuffle training data
        shuffle_test: whether to shuffle test data
  '''
  # Get train, test number
  num_total = len(dataset)
  num_train = int(num_total * 0.8 + 1)
  num_test  = num_total - num_train

  # random split dataset
  data_train, data_test = random_split(dataset, [num_train, num_test])

  # initialize dataloaders
  loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=shuffle_train)
  loader_test  = DataLoader(data_test, batch_size=batch_size, shuffle=shuffle_test)

  return loader_train, loader_test

### Part-2.1 Pretrain Network

Here we use a pretrain Neural Network to start with, then we fine tune it with the data set we have from **Adaptiope** in one domain, and test it on the target domain. Compare the two result, and set the benchmark for later UDA enriched method. 

In [ ]:
# pd_dataset = get_dataset(product_path)
# len(pd_dataset.classes)

### Part-2.2 Define the Model with Feature Extractor, Classifier, and Domain Regressor.

In [ ]:
# class DAAN(torch.nn.Module):
#   def __init__(self, num_classes):
#     super(DAAN, self).__init__()

#     # Feature Extractor with Pretained VGG16
#     # self.feature_extractor = vgg11(pretrained=True)

#     # Feature Extractor with AlexNet
#     self.feature_extractor = alexnet(pretrained=True)
#     feature_out_size = self.feature_extractor.classifier[-1].in_features
#     self.feature_extractor.classifier[-1] = torch.nn.Identity()

#     # Classifier
#     self.label_classifier = torch.nn.Linear(feature_out_size, num_classes)

#     # Domain Regressor
#     self.domain_regressor  = torch.nn.Linear(feature_out_size, 1)
#     self.domain_prediction = torch.nn.Sigmoid()

#   def forward(self, X):
#     # Feature Extractor
#     feature_representation = self.feature_extractor(X)

#     # Class Prediction
#     label_pred = self.label_classifier(feature_representation)

#     # Domain Regression Prediction
#     domain_rgrs = self.domain_regressor(feature_representation)
#     domain_pred = self.domain_prediction(domain_rgrs) 

#     # Concatenates label_prediction and domain_regression
#     # output = torch.cat((label_pred, domain_pred), dim=0)

#     return label_pred, domain_pred

In [ ]:
class FeatureExtractor(torch.nn.Module):
  def __init__(self):
    super(FeatureExtractor, self).__init__()

    # Feature Extractor with AlexNet
    self.feature_extractor = alexnet(pretrained=True)
    self.feature_dim = self.feature_extractor.classifier[-1].in_features

    # make the last layer identity
    self.feature_extractor.classifier[-1] = torch.nn.Identity()

  def forward(self, x):
    # Feature Extractor
    return self.feature_extractor(x)
  
  def output_dim(self):
    return self.feature_dim

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.classifier = torch.nn.Linear(input_dim, output_dim)
    
    def forward(self, X):
        return self.classifier(X) 

In [ ]:
from torch.autograd import Function

class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=256):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.discriminator = torch.nn.Linear(input_dim, hidden_dim)
        self.discrim_out   = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        intermed_out = F.relu(self.discriminator(x))
        output = torch.sigmoid(self.discrim_out(intermed_out))
        return output

In [ ]:
class DANN(torch.nn.Module):
  # def __init__(self, num_classes, adversarial=True):
  def __init__(self, num_classes):
    super(DANN, self).__init__()
    self.output_dim = num_classes

    # define inner network component
    self.feature_extractor = FeatureExtractor()
    self.classifier = Classifier(self.feature_extractor.output_dim(), num_classes)
    self.discriminator = Discriminator(self.feature_extractor.output_dim())  
  
  def forward(self, x):
    feature_output = self.feature_extractor(x)

    class_pred = self.classifier(feature_output)

    # Add a ReverseLayer here for negative gradient computation
    reverse_feature = ReverseLayerF.apply(feature_output, 1)
    discr_pred = self.discriminator(reverse_feature)

    return class_pred, discr_pred

### Part-2.2.1 Define the Feature Extractor by using a pretrain model

In [ ]:
# def feature_extractor():
#   '''
#   Using VGG16 pretrain model
#   And set the final classifer as an Identity Mapping(Use the model as feature extractor)
#   '''
#   model = vgg16(pretrained=True)
#   out_feature = model.classifier[-1].in_features
#   model.classifier[-1] = torch.nn.Identity()

#   return model, out_feature

### Part-2.2.2 Define the Classifier

In [ ]:
# def classifier(input_size, output_size):
#   return torch.nn.Linear(input_size, output_size)

### Part-2.2.3 Define the Domain Regressor

In [ ]:
# def domain_regressor(input_size, output_size=2):
#   return torch.nn.Linear(input_size, output_size)

### Part-2.3 Cost function

Divide parameters intro two groups, in which the last fully conneted layer with learning_rate, the other layers with 0.1 * learning_rate.

In [ ]:
def get_class_loss_func():
  return torch.nn.CrossEntropyLoss()

# def get_domain_loss_func():
  # return torch.nn.BCELoss()

### Part-2.4 Optimizer

In [ ]:
# def get_optimizer(model, learning_rate, weight_decay, momentum):
#   '''
#   Config Optimizer
#   '''
#   # classification layer name: label_classifier
#   classifier_name = "label_classifier"
#   domain_regressor_name = "domain_regressor"

#   pre_trained_weights = []
#   domain_regressor_weights = []
#   classifier_weights = []

#   # get all the parameters required gradient updates
#   for name, param in model.named_parameters():
#     if param.requires_grad == True:
#       if name.startswith(classifier_name):
#         classifier_weights.append(param)
#       elif name.startswith(domain_regressor_name):
#         domain_regressor_weights.append(param)
#       else:
#         pre_trained_weights.append(param)

#   # assign parameters to parameters
#   optimizer = torch.optim.SGD([
#     {'params': pre_trained_weights},
#     {'params': classifier_weights, 'lr': learning_rate}
#   ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
#   return optimizer

In [ ]:
def get_optimizer(model, config, adversarial=True):
  '''
  Config Optimizer
  '''
  learning_rate = config['lr']
  weight_decay  = config['wd']
  momentum      = config['momentum']

  feature_ext   = model.get_submodule("feature_extractor")
  classifier    = model.get_submodule("classifier")
  discriminator = model.get_submodule("discriminator")

  # classification layer name: label_classifier
  classifier_name = "label_classifier"
  domain_regressor_name = "domain_regressor"
  pre_trained_weights = feature_ext.parameters()

  if adversarial:
    other_weights = list(classifier.parameters()) + list(discriminator.parameters())
  else:
    other_weights = list(classifier.parameters())

  # assign parameters to parameters
  optimizer = torch.optim.SGD([
    {'params': pre_trained_weights},
    {'params': other_weights, 'lr': learning_rate}
  ], lr= learning_rate/10, weight_decay=weight_decay, momentum=momentum)
  
  return optimizer

### Part-2.5 Training and Testing Step

In [ ]:
def train_loop(dataloader, model, optimizer, device):
  size = len(dataloader.dataset)
  loss_fn = get_class_loss_func()

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    
    # compute prediction and loss
    class_pred, _ = model(X)

    # classification loss
    loss = loss_fn(class_pred, y)

    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
    
    del loss

In [ ]:
def test_loop(dataloader, model, device):
  test_loss, correct = 0, 0
  loss_fn = get_class_loss_func()

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      class_pred, _ = model(X)

      test_loss += loss_fn(class_pred, y).item()
      correct += (class_pred.argmax(1) == y).type(torch.float).sum().item()

  size = len(dataloader.dataset)
  num_batches = len(dataloader)

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

  return test_loss, correct

### Part-2.6 Training

In [ ]:
def training(model, train_dataloader, test_dataloader, config, device):
  epochs = config['epochs']
  print(f"Learning_rate {config['lr']}, weight_decay {config['wd']}")
  loss_fn = get_class_loss_func()

  optimizer = get_optimizer(model, config, adversarial=False)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    test_loop(test_dataloader, model, loss_fn, device)

  print("Done")

In [ ]:
# Get train_dataloader, test_dataloader
source_dataset = get_dataset(product_path)
train_dataloader, test_dataloader = get_dataloader(source_dataset, 50)

model = DANN(len(source_dataset.classes), False).to(device)

# Training
training(model, train_dataloader, test_dataloader, config, device)

Learning_rate 0.001, weight_decay 0.001
get_optimizer
FeatureExtractor(
  (feature_extractor): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


loss: 3.349539 [    0/ 1601]
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.335081 

Epoch 2
------------------
loss: 0.159048 [    0/ 1601]
Test Error: 
 Accuracy: 91.7%, Avg loss: 0.263871 

Epoch 3
------------------
loss: 0.211627 [    0/ 1601]
Test Error: 
 Accuracy: 93.0%, Avg loss: 0.245775 

Epoch 4
------------------
loss: 0.240699 [    0/ 1601]
Test Error: 
 Accuracy: 92.2%, Avg loss: 0.219472 

Epoch 5
------------------
loss: 0.052230 [    0/ 1601]
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.204437 

Done


### Part-2.7 Testing on Target Domain
#### Apply the model trained on the source domain directly to the target domain. This result will be used for comparison with the results obtained after domain adaptation.

In [ ]:
target_dataset = get_dataset(real_life_path)
loader_target_dataset = DataLoader(target_dataset, batch_size=100, shuffle=False)

# model.load_state_dict(torch.load('model_state.pt', map_location='cpu'))
test_loop(loader_target_dataset, model, device)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Error: 
 Accuracy: 63.7%, Avg loss: 1.203011 



(1.2030107021331786, 0.6375)

In [ ]:
del model
print(torch.cuda.memory_allocated())

940750848


In [ ]:
del train_dataloader, test_dataloader, loader_target_dataset

In [ ]:
print(torch.cuda.memory_allocated())

478516736


## TODO

TODO: Dataset unzip Google Drive, Copy to folder

TODO: Batch progress number error

TODO: Early stop or dropout, when accuracy doesn't improve much

Otherwise Continue UDA

## 3: MK-MMD 

MultiKernel Maximum Mean Discrepancy

## Part-4: Comparison & Discussion
Here we compare the test result from the direct method and the UDA method. 

## Part-5: Conclusion